# Clustering Crypto

In [22]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


## Deliverable 1: Preprocessing the Data for PCA

In [34]:
# Load the crypto_data.csv dataset.
df = pd.read_csv("Resources/crypto_data.csv", index_col = 0)
df.tail(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


In [35]:
# Keep all the cryptocurrencies that are being traded.
False_rows = df[df["IsTrading"]==False].index
df.drop(False_rows, inplace = True)


In [36]:
# Keep all the cryptocurrencies that have a working algorithm.
df.isnull().sum()

CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [37]:
# Remove the "IsTrading" column. 
df.drop(columns = "IsTrading", inplace = True)
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [38]:
# Remove rows that have at least 1 null value.
df = df.dropna()

In [39]:
# Keep the rows where coins are mined.
not_mined = df[(df["TotalCoinsMined"]=="NaN") | (df["TotalCoinsMined"]==0) ].index
df.drop(not_mined, inplace = True)

In [40]:
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [41]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_names = pd.DataFrame(df["CoinName"], index = df.index)
df_names.head()


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [42]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df.drop(columns = ["CoinName"], inplace = True)
df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [43]:
# Use get_dummies() to create variables for text features.
X=pd.get_dummies(df, columns = ["Algorithm", "ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
# print(X[0:5])

## Deliverable 2: Reducing Data Dimensions Using PCA

In [45]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [66]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data = crypto_pca, columns = ["PC 1", "PC 2", "PC 3"], index = df.index)
pcs_df.head(25)

,PC 1,PC 2,PC 3
42,-0.338422,1.005893,-0.598802
404,-0.321691,1.006180,-0.599211
1337,2.320291,1.713563,-0.654889
BTC,-0.144965,-1.310545,0.174258
ETH,-0.139185,-2.055346,0.405611
LTC,-0.167760,-1.150599,-0.012475
DASH,-0.397147,1.246557,-0.510545
XMR,-0.150544,-2.185598,0.414346
ETC,-0.137621,-2.055427,0.405605
ZEC,-0.124727,-2.044816,0.336883


## Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [67]:
inertia = []
k = list(range(1, 11))

In [68]:
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

C:\Users\awelsh\Anaconda3\envs\PyData\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [69]:
# Create an elbow curve to find the best value for K.
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [70]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=1)
model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
# print(predictions)

In [72]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df], axis = 1)
print(clustered_df.shape)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, df_names], axis = 1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 7)
(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.338422,1.005893,-0.598802,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321691,1.006180,-0.599211,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.320291,1.713563,-0.654889,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144965,-1.310545,0.174258,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.139185,-2.055346,0.405611,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.167760,-1.150599,-0.012475,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.397147,1.246557,-0.510545,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150544,-2.185598,0.414346,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.137621,-2.055427,0.405605,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.124727,-2.044816,0.336883,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [85]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x = "PC 1",
    y = "PC 2",
    z = "PC 3",
    color = "Class", 
    symbol = "Class",
    hover_name = "CoinName",
    hover_data=["Algorithm"], 
    width = 800,
)
fig.update_layout(legend = dict(x = 0, y=1))
fig.show()

In [87]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns = ["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [91]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {len(clustered_df)} tradable cryptocurrencies.")

There are 533 tradable cryptocurrencies.


In [92]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_data = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(scaled_data)

[[4.20000000e-11 5.94230127e-03]
 [5.32000000e-04 7.00182308e-03]
 [3.14159265e-01 3.53420682e-02]
 ...
 [1.40022261e-03 6.92655266e-03]
 [2.10000000e-05 5.94962775e-03]
 [1.00000000e-06 5.94243008e-03]]


In [93]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data = scaled_data, columns = ["TotalCoinSupply", "TotalCoinsMined"], index = clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = df_names["CoinName"].values

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"].values

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,1
404,5.320000e-04,0.007002,404Coin,1
1337,3.141593e-01,0.035342,EliteCoin,1
BTC,2.100000e-05,0.005960,Bitcoin,0
ETH,0.000000e+00,0.006050,Ethereum,0
LTC,8.400000e-05,0.006006,Litecoin,0
DASH,2.200000e-05,0.005951,Dash,1
XMR,0.000000e+00,0.005960,Monero,0
ETC,2.100000e-04,0.006056,Ethereum Classic,0
ZEC,2.100000e-05,0.005950,ZCash,0


In [94]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x = "TotalCoinsMined",
    y = "TotalCoinSupply", 
    by = "Class",
    hover_cols = "CoinName",
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)